## Contents

- [import necessary libraries](#0) 

-  feature extraction and MRC analysis

- 
  [1 - feature_extraction(class)](#1)
   !! includes MRC analysis

  
    calculates maximum results of parameters
    usage example: 
    fe = feature_extraction(stripped = stripped_nifti_file_path, original = original_nifti_file_path, 
                                 min_power = 1, max_power = 10, min_step = 1, max_step = 10, 
                                 ROI_shape = 'circle', 
                                primary_rate = 'all', secondary_rate = 'all',
                                all_powers = True, all_steps = True)
    fe.calculate_features(calculate_MRC = True)
    MRC_results = fe.MRC_results
    
    
- [2. additional functions for MRC analysis](#2)
    -    [2.1 take_rate_relation_list (function)](#2.1)
        gives a list which is spesify relationship for calculating rates    
    -    [2.2 take_pixel_numbers_of_kernel(function)](#2.2)
        gives pixel numbers within spesific ROI
    -    [2.3 reshape_arr(d\function)](#2.3)
        this function reshapes given array, 
        the function change axis by taking the smallest one to the beginning
        protect other axis order
    -    [2.3.1 reshape_arr_and_axis (function)](#2.3.1)
        the function is reshaping array and also giving new directions
    -    [2.4 rescale(function)](#2.4)
        rescale given array to the [0, 1] range
    -    [2.5 take_positive_minimum(function)](#2.5)
        gives positive minimum value of array, not zero, not nan
    -    [2.6 make_sequence(function)](#2.6)
        makes list of numbers 
    -    [2.7 take_csv_file_path(function)](#2.7)
        gives csf file for saving MRS results
    -    [2.8 update_special_file_list(function)](#2.8)
        checking if the given files proccessed or not 
    -    [2.9 take_collective_case_infos(function)](#2.9)
        gives cases info within collective calculations with plural planes of same sequences
    -    [2.10 take_pkl_file_path(function)](#2.10)
        gives specific pkl file path
    -    [2.11 check_if_MRC_completed(function)](#2.11)
        checks if MRC proccess completed 
    -    [2.12 save_results_to_csv(function)](#2.12)
        saves the data as csv file into output folder
    -    [2.13 run_apply_async_multiprocessing(function)](#2.13)
        run multiprocessing
      
- [3. EPE Analysis (class)](#3)
    making EPE analysis which explained in the article

<a name='0'></a>
###  import necessary libraries

In [5]:
import numpy as np
import os
import nibabel as nib
import pandas as pd
from sys import exit

from tqdm import tqdm
import multiprocessing
multiprocessing.set_start_method('fork') 
# multiprocessing.set_start_method('spawn') 
from multiprocessing.pool import Pool

<a name='1'></a>
### 1 - feature extraction and MRC analysis

In [5]:
def concat_2_df(df1, df2):
    
    if not df1.empty and not df2.empty:
        result_df = pd.concat([df1, df2], axis=0, ignore_index=True)
    elif not df1.empty:
        result_df = df1.copy()
    elif not df2.empty:
        result_df = df2.copy()
    else:
        result_df = pd.DataFrame()
    return(result_df)

In [1]:
class Feature_Extraction():
    
    """
    This class is used for calculating the features based on given parameters and 
        also for calculating the maximum results of each feature
        
    -- features are created based on specific functions and limits which explained in the article
    
    params:
    *# : explained in the article
    stripped (type : str (path_like)) :   stripped nifti file paths  
    original (type : str (path_like)) :   original nifti file path
    power (type : int)(*#) : power value for calculation power of primary rates 
    step  (type : int)(*#) : n value for calculation of kernel size 
    ROI_shape (type : str)(*#) : 'circle' or square : kernel(ROI) shape  
    primary_rate (type : str or int)(*#) :    
            'all' or value(1<= value <=3) 
            if primary_rate = 'all' - > all primary rates will be calculated
            if primary_rate = int. - > only corresponding primary rate will be calculated 
    secondary_rate (type : str or int)(*#) : 'all' or value(1<= value <=8) or value list
            if secondary_rate = 'all' - > all primary rates will be calculated
            if secondary_rate = int. - > only corresponding secondary rate will be calculated 
    
    all_powers (type : bool) : True: each power will be calculated, False: only given power will be calculated
    all_steps  (type : bool) : 
            True:  ROI will be calculated for each step,
            False: ROI will be calculated for only given step 
    
    
    """
    
    
    def __init__(self, stripped = 'path', original = 'path', 
                            min_power = 0, max_power = 0, power = 0,
                             min_step = 0, max_step = 0, step = 0,
                             ROI_shape = "square", 
                              primary_rate = 'all', secondary_rate = 'all',
                                 all_powers = False, all_steps = False):
        
        ## check given files if in nifti format
        self.check_if_file_nifti(stripped)
        self.check_if_file_nifti(original)
        
        ## assign inputs to objects for usage in class
        self.stripped_path = stripped
        self.original_path = original
        self.ROI_shape = ROI_shape
        self.primary_rate = primary_rate
        self.secondary_rate = secondary_rate
        self.power = power
        self.secondary_rate_list = []
        
        ## update secondary rate objects based on given input
        if type(secondary_rate) == list:
            self.secondary_rate_list = secondary_rate
            self.secondary_rate = max(secondary_rate)
         
        ##list specify final rates denominators (*#)
        self.sign_list = take_rate_relation_list() 
                       
        ## creating list for specify step and power limits  
        if all_powers:
            self.power_sq = make_sequence(max_power, min_power)
            
        if all_steps:
            self.step_sq = make_sequence(max_step, min_step)
        else:
            self.step_sq = [step]
            
        
        ## specify column names for storage MRC results
        self.columns = ['shape', 'primary_rate', 'secondary_rate', 'step', 'power', 'MRC_value']
        
    
    def calculate_features(self, calculate_MRC = False):
        
            
        """
        main function of the class
        this function is called for calculating MRC results

        output: 
        all_MRC_results (type : dictionary) : MRC results, keys : steps
         

        """
        
        self.calculate_MRC = calculate_MRC

        ####------- 1. loading data        
        strp_img = nib.load(self.stripped_path) ## loading stripped image 
        org_img = nib.load(self.original_path)  ## loading original image
        
        # getting arrays
        self.strp_data = strp_img.get_fdata() ## loading stripped image pixel values
        self.org_data = org_img.get_fdata()   ## loading original image pixel values

        ####-------- 2. Preprocessing of the data (*#)
        self.data = self.preprocessing_data()

         
        ####-------- 3. Processing of the Data (calculating MRC for per pixel and per feature) (*#)
                
        ## preparing denominators for primary rates
        ## i_0(AO1), i_MAD(AO2), i_MdAD(AO3)  (*#)
        self.core_denoms = self.take_denominators()
                                        
        ### start MRC dataframe for storage maximum of rates results for per feature if desired
        if self.calculate_MRC:
            self.MRC_results = pd.DataFrame(columns = self.columns)  
             
        for step_size in self.step_sq:
  
            ##---- 3.1 preparation of array for processing
            # taking pixel number of ROI for calculations
            self.kernel_pix_num = take_pixel_numbers_of_kernel(kernel_len = 2 * step_size + 1,
                                                               shape = self.ROI_shape)
            # creating 4 dimensional cluster for making calculations fastly
            self.cluster = self.make_kernel_cluster(step = step_size)
            
            ### object for storage each rates
            self.rates = {}    

            ##---- 3.2 calculating standart deviations (σ_all, σ_nbh) for primary rates
            self.std_all, self.std_ngb = self.std_calculation()

            ##----- 3.3 calculating difference between standart deviations : "σ_all- σ_nbh"
            self.std_diff = self.std_all - self.std_ngb
            
            ## 3.3.0- the pixel whose "σ_all- σ_nbh" < 0 means their harmony is better with together
            ## means: we don't need to calculate them             
            self.std_diff[self.std_diff <= 0] = np.nan

            ##------3.4 calculation MRC results of primary rates (rates without addditional operation) 
            self.rates['1'] = self.primary_rates_calculation()

            ## calculate MRC results for first rate            
            if self.calculate_MRC:
                self.calculate_MRC_results(step_size =step_size, rate_id = 1)

            ##------3.5 calculation MRC results of remained rates 
            ## limiting rates calculation based on input parameters           
            rates_lim = self.find_secondary_rates_limit()

            ## calculate rates and obtain maximums (MRC) (*#)    
            for j in range (2, rates_lim):
                self.rates[str(j)] =  self.rates_calculation(operation_number = j)
                ## calculate MRC results for other rates             
                if self.calculate_MRC:          
                    self.calculate_MRC_results(step_size =step_size, rate_id = j)


    def take_single_feature_list(self):
        
        """
        this function gives each pixels feature results in a list fromat without nan values
        - the function is usable for single feture calculations
        params:
        -
        (ufunciton use self.rates)
        
        outputs:
        feature_list(type: list) : contains features values from each valid pixel for proccess
        
        """
        
        ## since we calculated only 1 features, results will return in '1' key
        ## since we have only 1 primary rate and power, we will take 0.th axis        
        feature_arr = self.rates['1'][0][0].flatten()  
        ## remove nans fro taking results only from brain
        feature_list = feature_arr[np.logical_not(np.isnan(feature_arr))]
        
        return(feature_list)

                

    def calculate_MRC_results(self, step_size = 1, rate_id = 1):

        """
        This function call "take_MRC_results_df" function for calculation of MRC results 
        and store results to "self.MRC_results" dataframe 

        """

        if rate_id == 1:

            ## take MRC values and features dataframe for 1st secondary rate and all powers
            MRC_df = self.take_MRC_results_df(step_size, rate_id)

            ## adding to the main result dataframe
            self.MRC_results = concat_2_df(self.MRC_results, MRC_df)

        else:

            ## take MRC values and features dataframe for remained secondary rate and all powers
            MRC_df = self.take_MRC_results_df(step_size, rate_id)

            ## adding to the main result dataframe
            self.MRC_results = concat_2_df(self.MRC_results, MRC_df)


        
        

    #### function for the Preprocessing of the data
    def preprocessing_data(self):
        
        """
        this function make preprocess; prepare data to the process (*#)
        makes:
        1- reshaping : put 2 dimensional slice at the 1th and 2nd axis for calculations
                       img slices would be like : img[i, :, :] (i indicates slice number)
        2- rescaling (*#)
        
        3- maks non-brain pixels with 'nan'

        output: 
        data (type : array_like) : 3 dimensional array 
        """
        
        ##calculate min and max values of original image for rescaling
        min_org, max_org = np.nanmin(self.org_data), np.nanmax(self.org_data)
        
        ##  reshaping data to obtain spesific format for calculations 
        data, self.eliminated_coordinates_number = reshape_arr(self.strp_data) 
        
        ##  rescaling data to obtain spesific format for calculations 
        data = rescale(data, min_org, max_org)
        
        ## making 0 values 'nan' for discriminate non-brain values in further calculations
        ## note: 0 values represents non-brain pixels
        data[data == 0] = np.nan
        
        # deleting original image and data we will not use original data anymore
        del self.org_data, self.strp_data
        
        return(data)
            
    def find_secondary_rates_limit(self):
        
        """
        this function gives a number which is indicates limit for calculation final rates

        output: 
        rates_lim (type : int) : secondary rate limit
        """
        
        if self.secondary_rate == 'all' :
                rates_lim = 9 
        else:
            rates_lim = self.secondary_rate + 1
        return(rates_lim)
    
    def primary_rates_calculation(self):
        
        """
        this function calculate primary rates with desired powers  (*#)
        
  
        output: 
        p_rates (type : array_like) : 5 dimensional array which include primary rates with different powers
                    0 th axis: indicate primary rate (PR1, PPR2 or PR 3) 
                    1 th axis: indicate power of primary rate
                    3-4-5 : 3 Dimensional data  (primary rates with power)
        """
        
        shape = self.data.shape
        if self.primary_rate == 'all':
            p_rates = np.zeros((3, len(self.power_sq), shape[0], shape[1], shape[2]))  
            
            for i, p in enumerate(self.power_sq): 
                
                pw_std_all = np.power(self.std_all, p)
                pw_std_diff = np.power(self.std_diff, p) 
                p_rates[0][i] = np.divide(pw_std_all, self.std_ngb, out=np.zeros_like(pw_std_all), where=self.std_ngb!=0)      
                p_rates[1][i] = np.divide(pw_std_diff, self.std_all, out=np.zeros_like(pw_std_diff), where=self.std_all!=0)        
                p_rates[2][i] = np.divide(pw_std_diff, self.std_ngb, out=np.zeros_like(pw_std_diff), where=self.std_ngb!=0)
            
            del self.std_all, self.std_ngb  ## we don't need anymore to this objects
 
            return(p_rates)
        
        ## this part will be processed if primary_rate != 'all': for only choosen primary rate
        p_rates = np.zeros((shape[0], shape[1], shape[2])) 
        

        pw_std_all = np.power(self.std_all, self.power)
        pw_std_diff = np.power(self.std_diff, self.power) 

        if self.primary_rate == 1:
            p_rates = np.divide(pw_std_all, self.std_ngb, out=np.zeros_like(pw_std_all), where=self.std_ngb!=0) 
        elif self.primary_rate == 2:
            p_rates = np.divide(pw_std_diff, self.std_all, out=np.zeros_like(pw_std_diff), where=self.std_all!=0)    
        elif self.primary_rate == 3:    
            p_rates = np.divide(pw_std_diff, self.std_ngb, out=np.zeros_like(pw_std_diff), where=self.std_ngb!=0)
        else:
            print(f"you entered wrong argument for primary rate")
                
        del self.std_all, self.std_ngb  ## we don't need anymore to this objects
        
        return(p_rates)
    
    
    
    def take_MRC_results_df(self, step_size = 1,  rate_id = 1):
        
        """
        this function gives MRC results withing a dataframe which contain also features informations

        parameter:
            rate_id (type: int) : indicates secondary rate id
        outputs:
        MRC_df (type : pandas_DataFrame) : consist of MRC results 
                and feature informations based on self.columns
                
        self.columns = ['shape', 'primary_rate', 'secondary_rate', 'step', 'power', 'MRC_value']
        
        """

        ## calculating MRC by takign maximum values
        
        MRC_info = []
                
        if self.primary_rate == 'all' :            
            MRC_values = np.nanmax(self.rates[str(rate_id)], axis=(2,3,4), initial=0)        
            for i, p in enumerate(self.power_sq):                 
                MRC_info.append([self.ROI_shape, 1, rate_id, step_size, p, MRC_values[0][i]])
                MRC_info.append([self.ROI_shape, 2, rate_id, step_size, p, MRC_values[1][i]])
                MRC_info.append([self.ROI_shape, 3, rate_id, step_size, p, MRC_values[2][i]])        
            MRC_df = pd.DataFrame(MRC_info, columns = self.columns)
            return(MRC_df)
        
        else:            
            MRC_values = np.nanmax(self.rates[str(rate_id)], initial=0)            
            for i, p in enumerate(self.power_sq):                 
                MRC_info.append([self.ROI_shape, self.primary_rate, rate_id, step_size, p, MRC_values])                
            MRC_df = pd.DataFrame(MRC_info, columns = self.columns)                     
            return(MRC_df)
            
        
        
            
    def rates_calculation(self, operation_number = 0):
       
        """
        this function gives rates (final rates) results for obtaining MRC values 
        by using primary rates and denominators  (*#)
        
        params:
        operation_number(type : int) : this number is used for 
                                        identify which denominator and which primary rate will be used (*#)

        output: 
        s_rates (type : array_like) : 5 dimensional array
                        0 th axis: indicate primary rate (PR1, PPR2 or PR 3) 
                        1 th axis: indicate power of primary rate
                        3-4-5 : 3 Dimensional data  (final rates)
                        
        """
        
        self.clean_data(operation_number - 1 ) ## cleaning data which are not necessary anymore
        
        # s0 same with operation_number, s1: primary rate id, s2: denominator id
        s0, s1, s2 = self.sign_list[operation_number - 2] 
        shape = self.data.shape

        
        if self.primary_rate == 'all':
            s_rates = np.zeros((3, len(self.power_sq), shape[0], shape[1], shape[2]))               
            ## setting denominator based on data shape
            denom_4 = self.expand_3dim_array_to_4dim(self.core_denoms[s2], len(self.power_sq)) 
            denom_5 = self.expand_4dim_array_to_5dim(denom_4, 3)             
            ## calculating rate result by dividing primary rate(with appropriate powere) to denominator
            s_rates =  np.divide(self.rates[s1], denom_5, out=np.zeros_like(self.rates[s1]), where = denom_5!=0)     
            return(s_rates)
        
        ## this part will be processed if primary_rate != 'all':        
        s_rates = np.zeros((shape[0], shape[1], shape[2]))         
        denom = self.core_denoms[s2]         
        s_rates =  np.divide(self.rates[s1], denom, out=np.zeros_like(self.rates[s1]), where=denom!=0)
        return(s_rates)
        

    def std_calculation(self):
        
        
        """
        this function gives standard deviations of ROI for each pixel located at the center of the ROI (*#)
        

        output: 
        std_all (type : array_like) : standard deviation of each pixel within ROI (σ_all)
        std_ngb (type : array_like) : standard deviation of only neighbor pixels within ROI (σ_nbh)
                                      (σ_nbh : center pixel is not included)
                        
        """
                
        # calculation standart deviations (σ_all )throughout ROI(kernel)             
        std_all = np.std(self.cluster, axis = 1)      
        
        # for calculating  σ_nbh:         
        # calculate index of center pixel
        center_index = int ((self.kernel_pix_num - 1) / 2 )        
        # remove center pixel from cluster
        sq_ind = int((self.kernel_pix_num -1) / 2)        
        self.cluster = np.delete(self.cluster, sq_ind , axis = 1)       
        ### calculate naighbors standard deviations (σ_nbh )
        std_ngb = np.std(self.cluster, axis = 1)       
        del self.cluster ## we don't need cluster anymore        
        return(std_all, std_ngb)
      
        

            
    
    ### creating 4 dimensional array for further calculations   
    def make_kernel_cluster(self, step = 0):
        
        
        """
        this function created a cluster which is 4 dimensional, 
        first dimension same with pixel number in the ROI
        the cluster were used for making ROI calculations fastly, 
        at the 1th axis, each pixel ROI members were storage, 
        and the subject pixel is localized at the center
        
        params:
        step(type : int) (*#): n : specify kernel size ( = (kernel size - 1)) / 2 )

        output: 
        cluster (type : array_like) : 4 dimensional array

        """

        # creating initial arrays for procccess
        ## padding data for further calculations
        data_shape = self.data.shape
        kernel_len = step * 2 + 1
        pad_data = np.zeros((data_shape[0], kernel_len + 1 + data_shape[1],  kernel_len + 1 + data_shape[2]))
        pad_data[:, step+1 : -step-1,  step+1 : -step-1] = self.data
        
        ## 4- dimensional cluster will include all neighbors pixels in 1th axis and center pixels will be at the center        
        img_shape = self.data.shape
        cluster = np.zeros((img_shape[0], self.kernel_pix_num ,img_shape[1], img_shape[2]))     
        
        ## defining some limits for carrying data to padded data while matching neighbors with the same axis of the center pixel       
        if self.ROI_shape == "square":
            coordinates_dif = [(x,y) for x in range(-step, step+1) for y in range(-step, step+1)]
            coordinates_limits = [([step+x+1, -step+x-1], [step+y+1, -step+y-1]) for x,y in coordinates_dif]
            for i, limit in enumerate (coordinates_limits):            
                cluster[:, i, :, :] = pad_data[:, limit[0][0] : limit[0][1],  limit[1][0] : limit[1][1]]
            return(cluster)
        
        elif self.ROI_shape == "circle":
            coordinates_dif = [(x,y) for x in range(-step, step+1) for y in range(-step, step+1) if ( x**2 + y**2 ) <= step**2]
            coordinates_limits = [([step+x+1, -step+x-1], [step+y+1, -step+y-1]) for x,y in coordinates_dif]
            for i, limit in enumerate (coordinates_limits):            
                cluster[:, i, :, :] = pad_data[:, limit[0][0] : limit[0][1],  limit[1][0] : limit[1][1]]
            return(cluster)


        
        
    ### preparing denomitors for additional operations 
    
    def take_denominators(self):
        
        """
        this function creates 3 core denominator for calculating secondary rates (*#)

        output: 
        denom (type : array_like) : 3 dimensional array
        """

        denom = {}
        
        ## MAD, MdAD  calculation            
        median_value = np.nanmedian(self.data)    
        midrange_value = (np.nanmax(self.data) + np.nanmin(self.data)) / 2
        
        ## this denominator for intensity:  " i_0(AO1) " (*#)
        denom['0'] = np.copy(self.data)
        
        ## this denominator for median absolute deviation:  "i_MAD(AO2) " (*#)
        denom['1'] = np.abs(self.data - median_value)
        denom['1'][denom['1'] == 0] = take_positive_minimum(denom['1'])
               
        ## this denominator for midrange absolute deviation:  "i_MdAD(AO3) "  (*#)
        denom['2'] = np.abs(self.data - midrange_value)        
        denom['2'][denom['2'] == 0] = take_positive_minimum(denom['2'])
        
        return(denom)

 

        
    def clean_data(self, operation_number):    
        
        """
        this function deletes un-necessary objects for reduce memory           
        """       
        if operation_number == 4:
            if 4 not in self.secondary_rate_list:
                del self.rates['4']
        elif operation_number == 5 :
            if 1 not in self.secondary_rate_list:
                del self.rates['1']
        elif operation_number == 6 :
            if 6 not in self.secondary_rate_list:
                del self.rates['6']
        elif operation_number == 7 :
            if 2 not in self.secondary_rate_list:
                del self.rates['2']
            if 7 not in self.secondary_rate_list:
                del self.rates['7']
        elif operation_number == 8 :
            if 3 not in self.secondary_rate_list:
                del self.rates['3']
     
    def expand_3dim_array_to_4dim(self, array, added_size):
        """
        this function make 3 dimensional array 4 dimensional by repeating 3 dimensional array
        --the additional axis is added to the beginning
        --aim : making calculations in 4 dimension without using for loops
        
        output: 
        array(type: array_like) : 4 dimensonal array
        
        """
        if len(array.shape) == 3:        
            array = np.expand_dims(array, axis=0)
            array = np.tile(array,(added_size,1,1,1))
            return(array)
        return(print(f"given array is not 3 dimensional"))
    
    def expand_4dim_array_to_5dim(self, array, added_size):
        
        """
        this function make 4 dimensional array 5 dimensional by repeating 4 dimensional array
        --the additional axis is added to the beginning
        --aim : making calculations in 5 dimension without using for loops
        
        output: 
        array(type: array_like) : 5 dimensonal array
        
        """
        if len(array.shape) == 4:        
            array = np.expand_dims(array, axis=0)
            array = np.tile(array,(added_size,1,1,1,1))
            return(array)
        
        return(print(f"given array is not 4 dimensional"))
            
        
    ### control function for if nifti files paths showing if nifti files      
    def check_if_file_nifti(self, file_path):
        
        """
        this function check if file is nifti 
        
        params:
        file_path (type: string(path_like)) : file path
        
        output: 
        
        if file path is not showing nifti file, error message will be shown and
        proccess wil be ended with exit command
        
        """
        
        if_file = os.path.isfile(file_path)
        
        if '.nii.gz' not in file_path:
            print('the path indicated nifti files extension is not ".nii.gz"')
            if_file = False
            print(f' "{file_path}" path is not showing a nifti file')
            print(f"please check nifti files within the given path")
            exit()
            
 
 

<a name='2'></a>
### 2. additional functions for MRC analysis

<a name='2.1'></a>
#### 2.1

In [2]:
def take_rate_relation_list():
    """
    this function gives a list which is
    specify primary, additional operations (denominators) and secondary rates relations
    example:  ['2', '1', '0']: 
    '2' : Secondary Rate 2 will be calculated with follows: 
    '1' : gives secondary rate and this rate will be divided to
        following denominator for obtain Secondary Rate 2
    '0' : core denominator 0
    
    in the other words: 
    Secondary Rate 2  = Secondary Rate 1 / core denominator 0
    
    output:
    rates(type: list)
        
    """
    rates = []
    rates.append(['2', '1', '0']) # taking serie_2 by calculating first rates by dividing intensities
    rates.append(['3', '1', '1']) # taking serie_3 by calculating first rates by dividing distance to median
    rates.append(['4', '1', '2']) # taking serie_4 by calculating first rates by dividing distance to midrange
    rates.append(['5', '2', '1']) # taking serie_5 by calculating first rates by dividing intensity and distance to median
    rates.append(['6', '2', '2']) # taking serie_6 by calculating first rates by dividing intensity and distance to midrange
    rates.append(['7', '3', '2']) # taking serie_7 by calculating first rates by dividing distance to midrange and median
    rates.append(['8', '5', '2']) # taking serie_8 by calculating first rates by dividing intensity and distance to midrange and median
    return(rates)
    

<a name='2.2'></a>
#### 2.2

In [3]:
def take_pixel_numbers_of_kernel(kernel_len, shape = "square"):
    
    """
    this function gives pixel number within spesific ROI
    parameters:
    kernel_len(type: int) : ROI size:
        if shape      ==  "square"   : kernel_len = edge pixel number
        if shape == "circle"         : kernel_len = pixel number on diameter
     
    shape(type: str)  : ROI shape
    outputs:
    pixel_number(type:int) : pixel number within spesific ROI
    
    """
    
    if shape == "square" : 
        pixel_number = np.square(kernel_len)
        return(int(pixel_number))
    if shape == "circle":
        ## take radius size for circle ROI
        r = int((kernel_len - 1) / 2 )
        pixel_number = np.sum([1 for i in range (-r, r+1) for j in range (-r, r+1) if ( i**2 + j**2 ) <= r**2])
        return(int(pixel_number))

<a name='2.3'></a>
#### 2.3

In [4]:
def reshape_arr(img_arr):
    """
    this function reshapes given array, the function change axis by taking the smallest one to the beginning
    -- protect other axis order
    
    parameters:
    img_arr(type: numpy_array) : 3 dimensional numpy array

    outputs:
    new_arr(type: numpy_array) : 3 dimensional numpy array
    
    """


    
    img_shape = np.array(img_arr.shape)
    x, y, z = img_shape[0], img_shape[1], img_shape[2]

       
    min_slice = np.argmin(img_shape)


    img_shape[0] = img_shape[min_slice]
    img_shape[min_slice] = x
    if min_slice == 2:
        img_shape[1], img_shape[2] = x, y
    new_arr = np.zeros((img_shape[0], img_shape[1], img_shape[2]))
    for i in range (0, img_shape[0]):
        if min_slice == 0:
           new_arr[i, :, :] =  img_arr[i, :, :]
        elif min_slice == 1:
           new_arr[i, :, :] =  img_arr[:, i, :]
        else:
           new_arr[i, :, :] =  img_arr[:, :, i]
  
    
    arr_shape = new_arr.shape
    min0 = arr_shape[0]
    min1 = arr_shape[1]
    min2 = arr_shape[2]
    max0 = 0
    max1 = 0
    max2 = 0
    
    arr_0 = np.zeros((arr_shape[1], arr_shape[2]))    
    for i in range (0, arr_shape[0]):
        min0 = i
        if new_arr[i, :, :].any() != arr_0.any() :
            break
    for i in reversed(range(arr_shape[0])):
        max0 = i
        if new_arr[i, :, :].any() != arr_0.any() :
            break
            
    arr_1 = np.zeros((arr_shape[0], arr_shape[2]))    
    for i in range (0, arr_shape[1]):
        min1 = i
        if new_arr[:, i, :].any() != arr_1.any() :
            break
    for i in reversed(range(arr_shape[1])):
        max1 = i
        if new_arr[:, i, :].any() != arr_1.any() :
            break
            
    arr_2 = np.zeros((arr_shape[0], arr_shape[1]))    
    for i in range (0, arr_shape[2]):
        min2 = i
        if new_arr[:, :, i].any() != arr_2.any() :
            break
    for i in reversed(range(arr_shape[2])):
        max2 = i
        if new_arr[:, :, i].any() != arr_2.any() :
            break
            
    new_arr = new_arr[min0:max0+1, min1:max1+1, min2:max2+1]

    
            
    return(new_arr, [min0, min1, min2])


<a name='2.3.1'></a>
#### 2.3.1

In [5]:
def reshape_arr_and_axis(img_arr, axes):
    
    """
    This function is rotating an array in a specific way. 
    The function is reshaped array by making the array's 0th axis the smallest length of an array 
    which is actually for making slicing always at the 0th axis. 
    
    params:
    img_arr (type: numpy_array) : 3 dimensional numpy array
    axes (type: str) : (1, 3) dimensional array which is giving axis directions, such as ['A', 'S', 'L']
    
    outputs:
    new_arr(type: numpy_array) : 3 dimensional numpy array 
    new_axes(type: numpy_array) : (1, 3) dimensional array which is giving new axis directions
    
    """
    
    img_shape = np.array(img_arr.shape)
    x, y, z = img_shape[0], img_shape[1], img_shape[2]
    min_slice = np.argmin(img_shape)
    img_shape[0] = img_shape[min_slice]
    img_shape[min_slice] = x
    if min_slice == 2:
        img_shape[1], img_shape[2] = x, y
    new_arr = np.zeros((img_shape[0], img_shape[1], img_shape[2]))
    new_axes = ('m', 'm', 'm')
    for i in range (0, img_shape[0]):
        if min_slice == 0:
            new_arr[i, :, :] =  img_arr[i, :, :]
            new_axes = axes
        elif min_slice == 1:
            new_arr[i, :, :] =  img_arr[:, i, :]
            new_axes = (axes[1], axes[0], axes[2])            
        else:
            new_arr[i, :, :] =  img_arr[:, :, i]
            new_axes = (axes[2], axes[0], axes[1])
    
    return(new_arr, new_axes)


 
def reshape_arr_and_axis_affine(img_arr, axes, original_affine, zooms):
    """
    Reshapes a 3D array and updates the affine matrix to reflect non-isotropic voxel properties.
    
    params:
    img_arr (numpy array): 3D numpy array representing the image data.
    axes (tuple): Original axes directions (e.g., ('A', 'S', 'L')).
    original_affine (numpy array): Original 4x4 affine matrix of the NIfTI image.
    zooms (tuple): Voxel dimensions from the original NIfTI header.
    
    returns:
    new_arr (numpy array): Reshaped 3D numpy array.
    new_axes (tuple): New axes directions after reshaping.
    new_affine (numpy array): Updated 4x4 affine matrix preserving voxel size and spacing.
    """
    img_shape = np.array(img_arr.shape)
    min_slice = np.argmin(img_shape)  # Find the axis with the smallest length

    new_arr = img_arr
    new_affine = original_affine.copy()
    new_axes = axes

    if min_slice == 0:  # No change needed if the 0th axis is already the smallest
        new_arr = img_arr
    elif min_slice == 1:  # Move axis 1 to the front
        new_arr = np.transpose(img_arr, (1, 0, 2))
        new_affine[:3, :] = new_affine[[1, 0, 2], :]  # Swap rows 0 and 1
        # Adjust the voxel size scaling for the entire affine matrix
        new_affine[:3, :3] *= np.array([zooms[1], zooms[0], zooms[2]]) / np.array([zooms[0], zooms[1], zooms[2]])
        new_axes = (axes[1], axes[0], axes[2])
    elif min_slice == 2:  # Move axis 2 to the front
        new_arr = np.transpose(img_arr, (2, 0, 1))
        new_affine[:3, :] = new_affine[[2, 0, 1], :]  # Swap rows 0 and 2
        # Adjust the voxel size scaling for the entire affine matrix
        new_affine[:3, :3] *= np.array([zooms[2], zooms[0], zooms[1]]) / np.array([zooms[0], zooms[1], zooms[2]])
        new_axes = (axes[2], axes[0], axes[1])

    return new_arr, new_axes, new_affine





def update_affine_for_new_axes(original_affine, min_slice):
    """
    Adjusts the affine matrix based on the new axes order after reshaping.
    
    params:
    original_affine (numpy array): Original 4x4 affine matrix of the NIfTI image.
    min_slice (int): Index of the smallest axis used for reshaping.
    
    returns:
    new_affine (numpy array): Modified 4x4 affine matrix.
    """
    new_affine = original_affine.copy()
    
    # Adjust the affine matrix rows based on the new axes order
    if min_slice == 1:  # If the 1st axis becomes the 0th axis
        new_affine[:3, :] = new_affine[[1, 0, 2], :]
    elif min_slice == 2:  # If the 2nd axis becomes the 0th axis
        new_affine[:3, :] = new_affine[[2, 0, 1], :]
    
    return new_affine

<a name='2.4'></a>
#### 2.4

In [5]:
def rescale(data, min_value, max_value):
    
    """
    rescaling data to [0, 1] range by given minimum and maximum values
    
    params:
    data(type: numpy_array) : numpy array
    min_value(type: float/int) : spesific minimum value
    max_value(type: float/int) : spesific maximum value
    
    outputs:
    rescaled_data(type: numpy_array) : rescaled data between to [0, 1]
 
    """
    rescaled_data = (data - min_value) / (max_value - min_value)
    
    return (rescaled_data)

<a name='2.5'></a>
#### 2.5

In [6]:
def take_positive_minimum(data):
    """
    this function is giving a minimum value which is neither nan nor 0
    params : 
    data(type: numpy array) :numpy array
    outputs:
    minimum_value(type: float/ int) : positive minimum of data , not nan, not 0
    """
    
    data[data == 0] = np.nan
    minimum_value = np.nanmin(data)
    
    return(minimum_value)

<a name='2.6'></a>
#### 2.6

In [1]:
#2.6
def make_sequence(value_max = 0, value_min = 0):  
    
    """
    The function create sequence with a numbers between given params
    
    params:
    value_max(type: int) : maximum number for sequence
    value_min(type: int) : minimum number for sequence
    
    output:
    sequence(type: numpy array) : 1 dimensional array
        
    """

    if value_min > value_max:
        print('INPUT Error :')
        print('minimum value were given smaller then maximum value')
        return('ERROR')
    
    else:
        sequence = np.arange(value_min, value_max + 1)
        return(sequence)
                                 
    return(sequence)

<a name='2.7'></a>
#### 2.7

In [8]:
#2.7 
    
def take_csv_file_path(output_folder_path = '', original_file_path = '',
                        original_file_root_path =  ''):
    
    """
    this function gives csv file path for saving results
    parameters:
    
    output_folder_path (type: str(path_like)) : main folder path for saving MRC results 
    original_file_path (type: str(path_like)) : path of file which are used for MRC calculation
    original_file_root_path (type: str(path_like)) : root path of file for spesify 
                                            child and parent folder for saving csv files
                  
    output_path (type: str(path_like)) : gives csv path file for saving MRC results 
        
    """


    child_path = original_file_path.split(original_file_root_path)[-1][1:-7]
    output_path = os.path.join(output_folder_path, child_path + '_MRC.csv')
    
    return(output_path)

<a name='2.8'></a>
#### 2.8

In [5]:
def update_special_file_list(file_names_list, output_path = '', stripped_path =  ''):
    
    """
    this function is checking if the given files proccessed or not 
    
    params:
    file_names_list (type : list) :file name lists which will be proccessed 
    output_path(type: str : path-like) : folder path which includes proccessed files 
    stripped_path (type: str : path-like) : folder path which includes files to be processed 
    
    output:
    updated_list : files list which include only not processed files
    
    """

    updated_list = []    
    for file_names in file_names_list:       
        child_path = file_names[0].split(stripped_path)[-1][1:-7]
        csv_path = os.path.join(output_path, child_path + '_MRC.csv')
        if not os.path.isfile(csv_path):
            updated_list.append(file_names)
    return(updated_list)
    

<a name='2.9'></a>
#### 2.9

In [1]:
def take_collective_case_infos(situations, info_path, sequence_folders):

    """
    function gives cases info within collective calculations with plural planes of same sequences
    params:
    situations (type : array or list) : array includes situations keywords. example : ['patients', 'controls']
    info_path (type : str (path-like)) : excel file path which is consist of case-MRI data information
    sequence_folders(type : array or list) : includes sequence folder names. example ['t1', 't2']
    
    output:
    all_case_info (type: pandas dataframe) : includes case id and info and situation

    """

    all_case_info = pd.DataFrame()

    for situation in situations:
        ## reading info file
        file_ = pd.read_excel(info_path, sheet_name = None)
        ## taking sheet names
        sheet_names = list(file_.keys())
        ## filtering sheet names based on situation
        sheet_names = [sheet for sheet in sheet_names if situation in sheet]
        ## filtering sheet names depend on subfolders which are belong to same sequences
        sheet_names = [sheet for sheet in sheet_names for subfolder in sequence_folders if subfolder in sheet]
        ##create dataframe for loading infos
        info_df = pd.DataFrame(columns = ['input_name'])
        for sheet in sheet_names :
            df = file_[sheet][['input_name', 'case_name']]
            new_column_name = sheet.split(situation + '_')[1]
            df = df.rename(columns = {'case_name': new_column_name })
            info_df = pd.merge(info_df, df, how = "outer", on=['input_name'])
            info_df['situation'] = [situation] * len(info_df)
            info_df['case'] = ['coll_case' + str(i) for i in range (1, len(info_df) + 1)]
        all_case_info = pd.concat([info_df, all_case_info] , axis = 0, ignore_index=True)        
    return(all_case_info)


<a name='2.10'></a>
#### 2.10

In [11]:
def take_pkl_file_path(feature_vals, feature_output_folder):
    """
    gives specific pkl file path
    params:
    feature_vals(type: array or list) : features of pkl file 
    feature_output_folder(type: str) : folder path for saving the pkl file into
    output:
    file_path (type: str (path-like)) : the pkl file path for saving data

    """

    features_list = [str(feat) for feat in feature_vals] + ['.pkl']
    file_name = "-".join(features_list)
    file_path = os.path.join(feature_output_folder, file_name)
    return(file_path)  

<a name='2.11'></a>
#### 2.11

In [12]:
def check_if_MRC_completed(output_folder_path, original_file_path,
                    original_file_root_path, min_step, max_step):
    
    """
    the function checks if MRC proccess completed 
    
    params:
    output_folder_path(type : str (path-like)) : output folder path for MRC results
    original_file_path(type : str (path-like)) : original file path which MRC will be applied
    original_file_root_path(type : str (path-like)) : folder path which includes original files
    min_step(type : int) : minimum step for MRC calculation
    max_step(type : int) : maximum step for MRC calculation
    
    output:
    MRC_completed (type : bool) : True: MRC completed / False: MRC process not completed
    """
    
    MRC_completed = False
    csv_file_path = take_csv_file_path(output_folder_path, original_file_path, original_file_root_path)   
    is_file = os.path.isfile(csv_file_path)   
    if not is_file:       
        return(MRC_completed)
    else:
        data = pd.read_csv(csv_file_path)
        data_step_min = data['step'].min()        
        if min_step < data_step_min:
            return(MRC_completed)      
        data_step_max = data['step'].max()      
        if max_step > data_step_max:
            return(MRC_completed)
    MRC_completed = True
    return(MRC_completed)
    
    
    

<a name='2.12'></a>
#### 2.12

In [13]:
def save_results_to_csv(data, output_folder_path, original_file_path,
                    original_file_root_path):
    
    """
    saves the data as csv file into output folder
    params:
    data (type: pandas dataframe)    : pandas dataframe for saving to csv file
    output_folder_path(type : str (path-like)) : folder path for saving csv file into
    original_file_path(type : str (path-like)) : original file path for creating csv file name
    original_file_root_path(type : str (path-like)) : original files folder path for creating csv file name

    """
    
    csv_file_path = take_csv_file_path(output_folder_path, original_file_path, original_file_root_path)    
    if os.path.isfile(csv_file_path):        
        old_data = pd.read_csv(csv_file_path)       
        new_data = pd.concat([old_data, data], join='outer', ignore_index=True)     
        new_data.to_csv(csv_file_path, index=False, index_label = False)      
    else:      
        data.to_csv(csv_file_path, index=False, index_label = False)  

<a name='2.13'></a>
#### 2.13

In [ ]:
## function for making multipl proccesses at same time

def run_apply_async_multiprocessing(func, argument_list, num_processes):
    
    """
    run multiprocessing 
    params: 
    func (type: function): function which will be run
    argument_list (type: list): arguments list will be used in a function
    num_processes (type: int): number of multipl proccess
    
    """

    pool = Pool(processes=num_processes)

    jobs = [pool.apply_async(func=func, args=(*argument,)) if isinstance(argument, tuple) else pool.apply_async(func=func, args=(argument,)) for argument in argument_list]
    pool.close()
    
    result_list_tqdm = []
    for job in tqdm(jobs):
        result_list_tqdm.append(job.get())    
    return result_list_tqdm
    

<a name='3'></a>

## 3. EPE Analysis

In [5]:
class EPE_analysis():

    """   
    EPE analysis : Each point evaluation analysis
    This class is for making EPE analysis which explained in the article
    
    params for initialization :
    control_id (type: str) : general id word of control cases
    patient_id (type: str) : general id word of patients
    PCVL (type: str) : " per case value limit " / number of results for calculation per case 
    ** this limitation is used for decreasing coding time and memory 
    feature_output_folder (type: str (path-like)) : folder path for saving results
    follow_proccess (type: bool) :  False : Don't give files names information continually
                                    True: give files names information continually
    
    
    """

    
    def __init__(self, control_id, patient_id, PCVL  = 1000,
                 feature_output_folder = '', follow_proccess = False):
        
        self.control_id = control_id
        self.patient_id = patient_id
        self.feature_output_folder = feature_output_folder
        self.follow_proccess = follow_proccess
        self.PCVL = PCVL # per case value limit
    
                     
    def   activate_EPE_analysis(self, feature_info, 
                                controls_file_paths,  patients_file_paths,
                                specificity_min = 0.7, sensitivity_min = 0.7,
                                activate_collective_analysis = False, 
                                info_path = '', sequence_folders = []):
        
        """
        params:
        feature_info (type: pandas data frame) : parameters' features infos 
        controls_file_paths (type: str (path-like)) : coltrol cases files paths
        patients_file_paths (type: str (path-like)) : patients files paths
        specificity_min (type: float) : minimum specifity for selecting parameters
        sensitivity_min (type: float) : minimum sensitivity for selecting parameters
        activate_collective_analysis (type: bool) : True: the calculation is collective, 
                                                    False: the calculation isnot collective analysis
        info_path (type: str (path-like)) : Excel path which is consisting case matching informations
                                            This argument necessary for collective analysis
        sequence_folders (type: list or array) : sequence folders list 
                
        """
        
        
        ## specificity info for collective analysis
        self.activate_collective_analysis =  activate_collective_analysis     
        self.info_path, self.sequence_folders = info_path, sequence_folders

        ## specificity F1 limit for analysis
        self.specificity_min, self.sensitivity_min = specificity_min, sensitivity_min 
                        
        ## list for record stat results
        self.stat_results = []   

        ## specificity ROI shape for calculating seperately
        ROI_shapes = list(feature_info['shape'].unique())  
        
        for roi_shape in ROI_shapes:      
            
            features = feature_info[feature_info['shape'] == roi_shape]         
            
            ## specificity step sizes
            step_sizes = features['step'].unique()
            
            ## take feature info for per step size:
            for step in step_sizes:
                
                features_s = features[features['step'] == step]
                self.extract_features(features_s, roi_shape, controls_file_paths, patients_file_paths)
                
                ## run stat analysis                
                self.calculate_threshold_stats()

        ## transform results lists to dataframe:        
        new_column_names = list(feature_info.columns)[:-2] 
        new_column_names += ['AUC(MRC)', 'p(MRC)', 'threshold', 'specificity(EPE)', 'sensitivity(EPE)', 'EF_number_controls', 'EF_number_patients']
        result_df = pd.DataFrame(data = self.stat_results, columns = new_column_names)
        
        return(result_df)
    
    
    def extract_features(self, features, roi_shape, controls_file_paths, patients_file_paths):
        
        """
        print('extracting features)
        
        """
        
        ## create list for store features :
        
        self.feat_val_list = []

    
        ##---------------- deside single feature analysis or bulk analysis will be done

        # for single feature ---->             
        if len(features) == 1 :            
            # check_features_existence:            
            if self.feature_exist(features):                
                if self.follow_proccess == 'all':
                    print(f" features : {features} were read from existed file ")                
                return(0)

            control_data = self.single_calculation(features.iloc[0, :], controls_file_paths, roi_shape)
            patient_data = self.single_calculation(features.iloc[0, :], patients_file_paths, roi_shape)            
            control_data['situation'] = [self.control_id] * len(control_data)
            patient_data['situation'] = [self.patient_id] * len(patient_data)           
            feat_result = pd.concat([control_data, patient_data], ignore_index=True)          
            self.save_extracted_features(feat_result, list(features.iloc[0, :-2].values))       
            self.feat_val_list.append([features.iloc[0, :], feat_result])     
            del feat_result, control_data, patient_data   ## clean objects

        # for multipl features ---->
        elif len(features) > 1 :
            #check if features proccessed and saved before:           
            if self.feature_exist(features) : 
                if self.follow_proccess == 'all':
                    print(f" features : {features} were read from existed files ")            
                return(0)
            # extract features
            self.control_datas = self.multipl_calculation(features, controls_file_paths, roi_shape)
            self.patient_datas = self.multipl_calculation(features, patients_file_paths, roi_shape)            
            feat_num = -1 ## for double check features matching            
            while self.patient_datas:                
                feat_num += 1
                feat0 = features.iloc[feat_num, :]
                feat_control, control_data = self.control_datas[0]
                feat_patient, patient_data = self.patient_datas[0]                
                self.control_datas.pop(0), self.patient_datas.pop(0)  ## clean data
                
                ## checking if we are making counting for same features
                if  (feat0.values == feat_control.values).all() and (feat_control.values == feat_patient.values).all() :
                    
                    control_data['situation'] = [self.control_id] * len(control_data)
                    patient_data['situation'] = [self.patient_id] * len(patient_data)
                    feat_result = pd.concat([control_data, patient_data], axis = 0, ignore_index=True)                    
                    self.save_extracted_features(feat_result, list(features.iloc[feat_num, :-2].values))                                    
                    ## store multipl features and values dataframes to list
                    self.feat_val_list.append([feat0, feat_result])                     
                    del feat_result ## clean object


    
    
    def multipl_calculation(self, features, file_paths, roi_shape):



        """
        
        """
        
        all_feat_and_values = [] ## list for store values(dataframes) with their features(list)                        
        ## create dataframes for storage selected features values       
        for feat_num in range(len(features)):
            all_feat_and_values.append([ features.iloc[feat_num, :], pd.DataFrame()])      
        ## limiting calculation since we need to calculate untill max. power or step
        pw_min, pw_max = features['power'].min(), features['power'].max()     

        ## specificity sr list
        s_r_list = list(features['secondary_rate'].unique())
        s_r_list = [int(sr) for sr in s_r_list]   
        ## specificity step size
        step = features['step'].iloc[0]
        for file_names in file_paths:      
            if self.follow_proccess == 'all':
                print(f" '-- {file_names[0]} --'  is being processed ") 
            fe = Feature_Extraction(stripped = file_names[0], original = file_names[1], 
                        min_power = pw_min,  max_power = pw_max,
                        step = step,
                        ROI_shape = roi_shape, 
                        primary_rate = 'all', secondary_rate = s_r_list,
                        all_powers = True, all_steps = False)   
            
            fe.calculate_features()         
            eliminated_coordinates_numbers = fe.eliminated_coordinates_number ## get numbers to add coordinates
            case_id = self.extract_case_id(file_names[0])
            
            for feat_num in range(len(features)):      
                
                ## specificity feature infos
                feature_ = features.iloc[feat_num, :]
               
                ## get values from fe object
                pw, p_r, s_r =  feature_['power'],  feature_['primary_rate'], feature_['secondary_rate'] 
                feature_values = fe.rates[str(s_r)][p_r-1][pw - pw_min]                
                
                ##create dataframe with values and coordinates
                x,y,z = np.indices(feature_values.shape)  ## specificity indices for  coordinates


                # create dataframe with values and coordinates
                df = pd.DataFrame(np.c_[x.ravel(), y.ravel(), z.ravel(), feature_values.ravel()], columns=((['x','y','z', 'value'])))
                df = df.dropna(how='any') ## drop nan values

                ### convert coordinates to real values
                df[['x', 'y', 'z']] = df[['x', 'y', 'z']] + np.tile(eliminated_coordinates_numbers, (len(df),1))

                ## convert x-y-z columns to coordinates column by using tuple
                df['coordinates'] = df[['x', 'y', 'z']].apply(tuple, axis=1)

                ## remove x-y-z columns
                df = df[['value', 'coordinates']]

                ## add case id to dataframe
                df['case'] = [case_id] * len(df)
                
                ## order dataframe depend on value and take only maximum PCVL ( per case value limit)
                df.sort_values(by = 'value', axis=0, ascending=False, inplace=True,  ignore_index=True)
                if self.PCVL < len(df) :
                    df = df.iloc[:self.PCVL , :]
                
                ## unite dataframe with other cases dataframes
                all_feat_and_values[feat_num][1] = pd.concat([df, all_feat_and_values[feat_num][1]], ignore_index=True)
 
            del fe  ## clean object
    
        return(all_feat_and_values)
    

    def  single_calculation(self, feature_info, file_paths, roi_shape) :    

        
        """


        """
        
        if self.follow_proccess == 'all':
            print(f" feature extraction being procced for following files :  ") 

        feat_vals = pd.DataFrame(columns = ['case', 'value']) 
        pw, step = feature_info['power'], feature_info['step']
        p_r, s_r = feature_info['primary_rate'], feature_info['secondary_rate']
        
        for file_names in file_paths:                   
            if self.follow_proccess == 'all':
                print(f" '-- {file_names[0]} --'  is being processed ") 
            fe = Feature_Extraction(stripped = file_names[0], original = file_names[1], 
                            power = pw, step = step, ROI_shape = roi_shape, 
                            primary_rate = p_r, secondary_rate = s_r)            
            fe.calculate_features() ## calculate per features and MRC of features
            eliminated_coordinates_numbers = fe.eliminated_coordinates_number ## get numbers to add coordinates
            ## get feature array
            feature_values = fe.rates[str(s_r)]
            ##create dataframe with values and coordinates
            x,y,z = np.indices(feature_values.shape)  ## specificity indices for  coordinates    

            
            # create dataframe with values and coordinates
            df = pd.DataFrame(np.c_[x.ravel(), y.ravel(), z.ravel(), feature_values.ravel()], columns=((['x','y','z', 'value'])))            
            df = df.dropna(how='any') ## drop nan values     
            
            ### convert coordinates to real values
            df[['x', 'y', 'z']] = df[['x', 'y', 'z']] + np.tile(eliminated_coordinates_numbers, (len(df),1))
        
            ## convert x-y-z columns to coordinates column by using tuple
            df['coordinates'] = df[['x', 'y', 'z']].apply(tuple, axis=1)  
            
            ## remove x-y-z columns
            df = df[['value', 'coordinates']]           
            ## specificity case id                 
            case_id = self.extract_case_id(file_names[0])
            ## add case id to dataframe
            df['case'] = [case_id] * len(df)
            ## order dataframe depend on value and take only maximum PCVL ( per case value limit)
            df.sort_values(by = 'value', axis=0, ascending=False, inplace=True,  ignore_index=True)
            if self.PCVL < len(df) :
                df = df.iloc[:self.PCVL , :]
            ## unite dataframe with other cases dataframes
            feat_vals = pd.concat([feat_vals, df], ignore_index=True)
            
            del fe ## clean objects

        return(feat_vals)
    

  
   
                

        
    def   calculate_threshold_stats(self):
        
        
        """
        
        """
        
        ## check if collective analysis will be made
        if self.activate_collective_analysis : 
            
            coll_case_infos =  take_collective_case_infos([self.control_id, self.patient_id],
                                                               self.info_path, self.sequence_folders)
            
            self.coll_control_info = coll_case_infos[coll_case_infos['situation'] == self.control_id]
            self.coll_patient_info = coll_case_infos[coll_case_infos['situation'] == self.patient_id]
        

        ## calculating stats for per feature
        
        while self.feat_val_list:
            
            ## assign objects
            self.feature, self.value_df = self.feat_val_list[0]
            
            ## clean unnecessary object
            self.feat_val_list.pop(0)
                         
            ## update dataframe for collective analysis if necessary
            if self.activate_collective_analysis : 
                self.activate_collective_analysis_()
            
            
            ## define threshold list for values:           
            self.values = self.value_df['value'].values
            
            thresold, specificity, sensitivity, EF_controls, EF_patients = self.find_cut_off()
            

            if specificity >= self.specificity_min:
                
                self.stat_results.append(list(self.feature.values) + [thresold, specificity, sensitivity, EF_controls, EF_patients])
                
            if self.follow_proccess != 'none' :
                if self.follow_proccess == 'feature_and_stat':
                    print(f" feature parameters :  {self.feature}")
                if specificity >= self.specificity_min:
                    print(f" specificity score: {specificity}, sensitivity score: {sensitivity}")
                else:
                    print('scores were not found above desired limits')

            

    def activate_collective_analysis_(self):
        
        """
        
        
        """
        
        ## updating control case infos:
  
        for case_num in range(len(self.coll_control_info)):
            row = self.coll_control_info.iloc[case_num, :]
            case_id_list = [folder + '-' + row[folder] for folder in self.sequence_folders if type(row[folder]) == str ]            
            for case_id in case_id_list:        
                self.value_df['case'] = np.where((self.value_df['case'] == case_id)  & (self.value_df['situation'] == self.control_id), row['case'], self.value_df['case']  )
     
    
        ## updating patients case infos :
        
        for case_num in range(len(self.coll_patient_info)):
            row = self.coll_patient_info.iloc[case_num, :]
            case_id_list = [folder + '-' + row[folder] for folder in self.sequence_folders if type(row[folder]) == str ]            
            for case_id in case_id_list:        
                self.value_df['case'] = np.where((self.value_df['case'] == case_id)  & (self.value_df['situation'] == self.patient_id), row['case'], self.value_df['case']  )
     
    
        
    def find_cut_off(self):    
        """
        
        """ 

        ## create threshold list with standart deviation step size
        threshold_list = np.linspace(np.quantile(self.values, 0.9), np.nanmax(self.values), 1000)
        
        threshold_number = len(threshold_list)

        ## create dataframe with same values columns
        values_arr = np.tile(self.values.reshape(-1, 1), (1, threshold_number))

        ## transform to dtaframe for making calculations           
        thresholds_id = list(np.arange(threshold_number))
        df = pd.DataFrame(data = values_arr, columns = thresholds_id)

        ## filtering dataframe based on thresholds
        df[df <= threshold_list] = 0
        df[df > threshold_list] = 1

        ## place case ids and situations to indicate positive results owners
        df[['situation', 'case']] = self.value_df[['situation', 'case']]

        ## seperate controls and patients
        df_patients = df[thresholds_id + ['case']][df['situation'] == self.patient_id]
        df_controls = df[thresholds_id + ['case']][df['situation'] == self.control_id]
        
        
        ##  patients cases numbers
        patient_num, control_num = df_patients.nunique()['case'], df_controls.nunique()['case']

        ## specificity true positives and false positives
        TP, FP = self.take_positive_cases_number(df_patients), self.take_positive_cases_number(df_controls)
        TN, FN = control_num - FP, patient_num - TP

        #np.divide(pw_std_all, self.std_ngb, out=np.zeros_like(pw_std_all), where=self.std_ngb!=0)
        stat = pd.DataFrame()
        stat['sensitivity'] = np.divide(TP.astype(float) , (TP+FN).astype(float) , out=np.zeros_like(TP.astype(float)), where=(TP+FN)!=0)
        stat['specificity'] = np.divide(TN.astype(float) , (TN+FP).astype(float) , out=np.zeros_like(TN.astype(float)), where=(TN+FP)!=0)

        ## Eliminate rows that do not correspond to the desired sensitivity limit
        stat.loc[stat['sensitivity'] < self.sensitivity_min, 'specificity'] = 0 

        maxValueIndex = stat['specificity'].idxmax()
        thresold = threshold_list[maxValueIndex]
        specificity, sensitivity = stat['specificity'].iloc[maxValueIndex], stat['sensitivity'].iloc[maxValueIndex]
        EF_controls, EF_patients = df_controls[maxValueIndex].sum(), df_patients[maxValueIndex].sum()
        
        return(thresold, specificity, sensitivity, EF_controls, EF_patients)


    
    def take_positive_cases_number(self, data_df):
        
        """
        
        """
        if len(data_df) == 0:
            return(0)
        
        ## find positives
        data_df = data_df.groupby(by = 'case').sum()
        ## positive cases will be 1
        data_df[data_df > 0] = 1
        ## find number of positive cases per threshold:
        positive_case_numbers = data_df.sum(axis = 0).values
        
        return(positive_case_numbers)
    
    
    
    def extract_case_id(self, file_path):
    
        """
        This function gives case id of nifti file by extracting from file path

        parameters:
        file_path (type: string (path_like)) : nifti file path (file extension : .nii.gz)

        outputs:
        case_id (type : str) : case id

        """
        
        if self.activate_collective_analysis:
            splitted_file = os.path.split(file_path)
            case_id = splitted_file[1].split('.nii.gz')[0]
            folder_name = os.path.split(splitted_file[0])[1]
            coll_case_id = folder_name + '-' + case_id
            return(coll_case_id)

        file_name = os.path.split(file_path)[1]
        case_id = file_name.split('.nii.gz')[0]
        return(case_id)
    
    
    def save_extracted_features(self, data, features_list):
        """

        """
        
        features_list = [str(feat) for feat in features_list] + ['.pkl']
        file_name = "-".join(features_list)

        output_file_path = os.path.join(self.feature_output_folder, file_name)
        data.to_pickle(output_file_path) 
    
    
    def feature_exist(self, features):
        """
        """
        
        file_exist = False
        
        ## check if single feature list given or multipl features were given
        
        if len(features) ==  1:
            ## single feature were given:
            file_path = self.take_pkl_file_path(features.iloc[0, :-2].values)
            if os.path.isfile(file_path) :            
                self.feat_val_list = [[features.iloc[0, :], pd.read_pickle(file_path)]]
                file_exist = True                
            return(file_exist)

        elif len(features) > 1:            
            ## multipl features we given 
            file_paths = list(map(self.take_pkl_file_path, features.iloc[:, :-2].values))             
            not_exist_features = [i for i, file_path in enumerate(file_paths) if not os.path.isfile(file_path)]
            
            if not_exist_features:                
                return(file_exist)
                            
            self.feat_val_list = [[features.iloc[i, :], pd.read_pickle(file_path)] for i, file_path in enumerate(file_paths) if os.path.isfile(file_path)]            
            
            if self.feat_val_list :                
                file_exist = True

            return(file_exist)
        

                 
    def take_pkl_file_path(self, feature_vals):
        """
        
        """
        
        features_list = [str(feat) for feat in feature_vals] + ['.pkl']
        file_name = "-".join(features_list)
        file_path = os.path.join(self.feature_output_folder, file_name)
        
        return(file_path)  